In [28]:
import sqlite3
import pandas as pd
import json
import requests
import os
import time

## Création BDD et tables

In [13]:
try:
    db = sqlite3.connect("foil.db")
    
    cursor = db.cursor()
    
    cursor.execute('''create table stations (       
    
    id_station integer primary key,
    name varchar,
    latitude float,
    longitude float)''')
    
    cursor.execute('''create table mesures (       
    
    id_mesure integer primary key autoincrement unique,
    date varchar,
    id_station integer,
    pressure float,
    wind_heading float,
    wind_speed_avg float,
    wind_speed_max float,
    wind_speed_min float,
    status_state varchar,
    foreign key (id_station) references stations(id_station))''')
                                                     
    db.commit()
    
    cursor.close()
    db.close()
except:
    print("Une erreur est survenue lors de la creation de la base")
    exit(1)
    
print("La base de données a été créée")  

La base de données a été créée


## Création liste Stations et Mesures

In [2]:
liste_stations = [113, 307, 308]

In [3]:
data_stations=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_stations.append([response_info['data']['id'], response_info['data']['meta']['name'], response_info['data']['location']['latitude'], response_info['data']['location']['longitude']])

In [4]:
data_stations

[[113, 'Brétignolles sur mer - La Normandelière', 46.613681, -1.861328],
 [307, 'Pioupiou 307', 46.450667, -1.581897],
 [308, 'Pioupiou 308', 46.692604, -1.765012]]

In [17]:
data_mesures=[]
for i in liste_stations:
    q=str(i)
    response = requests.get('http://api.pioupiou.fr/v1/live/'+q)
    response_info = json.loads(response.text)
    data_mesures.append([response_info['data']['measurements']['date'], response_info['data']['id'], response_info['data']['measurements']['pressure'], response_info['data']['measurements']['wind_heading'], response_info['data']['measurements']['wind_speed_avg'], response_info['data']['measurements']['wind_speed_max'], response_info['data']['measurements']['wind_speed_min'], response_info['data']['status']['state']])

In [18]:
data_mesures

[['2021-03-11T22:14:30.000Z', 113, None, 270, 29.5, None, None, 'on'],
 ['2021-03-11T22:12:43.000Z', 307, None, 270, 13, 20.5, 7, 'on'],
 ['2021-03-11T22:07:25.000Z', 308, None, 22.5, 0, 0, 0, 'on']]

## Connexion BDD

In [7]:
conn = sqlite3.connect('foil.db')

## Insertion données

In [19]:
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

In [9]:
sql_inserer_station = """
    INSERT INTO stations
    (id_station, name, latitude, longitude)
    VALUES (?, ?, ?, ?)
"""

In [10]:
inserer_donnees(conn, sql_inserer_station, data_stations)

Erreur lors de l'insertion des données
UNIQUE constraint failed: stations.id_station


In [20]:
sql_inserer_mesure = """
    INSERT INTO mesures
    (date, id_station, pressure, wind_heading, wind_speed_avg, wind_speed_max, wind_speed_min, status_state)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""

In [21]:
inserer_donnees(conn, sql_inserer_mesure, data_mesures)

Les données ont été insérées avec succès


In [ ]:
while True:
    inserer_donnees(conn, sql_inserer_mesure, data_mesures)
    time.sleep(60)

Les données ont été insérées avec succès
Les données ont été insérées avec succès
Les données ont été insérées avec succès
Les données ont été insérées avec succès
Les données ont été insérées avec succès
Les données ont été insérées avec succès


In [12]:
sql = '''
SELECT *
FROM stations'''

sql = pd.read_sql_query(sql, conn)
sql

,id_station,name,latitude,longitude
0,113,Brétignolles sur mer - La Normandelière,46.613694,-1.861316
1,307,Pioupiou 307,46.450667,-1.581897
2,308,Pioupiou 308,46.692344,-1.764994


In [ ]:
sql = '''
SELECT *
FROM mesures'''

sql = pd.read_sql_query(sql, conn)
sql